In [ ]:
'''
from google.colab import drive
drive.mount('/content/gdrive')'''

Mounted at /content/gdrive


In [ ]:
'''
import os
path="/content/gdrive/My Drive"
os.chdir(path)
os.listdir(path)'''

['net_config (6).cpython-37.pyc',
 '.ipynb_checkpoints',
 'net_config (5).cpython-37.pyc',
 'EnsembleModel (1).cpython-37.pyc',
 'net_config (4).cpython-37.pyc',
 'EnsembleModel.cpython-37.pyc',
 'net_config (3).cpython-37.pyc',
 'seqGAN',
 'ML_dialogue',
 'seqGAN_tran.ipynb',
 '6000J',
 '5002',
 '5012',
 'exam5003',
 'net_config (2).cpython-37.pyc',
 'MultiLabelTextClassification',
 'net_config (1).cpython-37.pyc',
 'net_config.cpython-37.pyc',
 'MAAL.ipynb',
 '5002final',
 'MSBD5002_Fall2021_Final_Data']

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import math
import torch
import matplotlib.pyplot as plt
import math
from scipy.signal import find_peaks
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler


In [ ]:
confirmed_global = pd.read_table("MSBD5002_Fall2021_Final_Data/Q2/covid19_confirmed_global.txt",sep=',')
deaths_global = pd.read_table("MSBD5002_Fall2021_Final_Data/Q2/covid19_deaths_global.txt",sep=',')

In [ ]:
confirmed_global

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,10/21/21,10/22/21,10/23/21,10/24/21,10/25/21,10/26/21,10/27/21,10/28/21,10/29/21,10/30/21,10/31/21,11/1/21,11/2/21,11/3/21,11/4/21,11/5/21,11/6/21,11/7/21,11/8/21,11/9/21,11/10/21,11/11/21,11/12/21,11/13/21,11/14/21,11/15/21,11/16/21,11/17/21,11/18/21,11/19/21,11/20/21,11/21/21,11/22/21,11/23/21,11/24/21,11/25/21,11/26/21,11/27/21,11/28/21,11/29/21
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,5,5,...,155891,155931,155940,155944,156040,156071,156124,156166,156196,156210,156250,156284,156307,156323,156363,156392,156397,156397,156397,156397,156414,156456,156487,156510,156552,156610,156649,156739,156739,156812,156864,156896,156911,157015,157032,157144,157171,157190,157218,157260
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,180029,180623,181252,181696,181960,182610,183282,183873,184340,184887,185300,185497,186222,186793,187363,187994,187994,189125,189355,190125,190815,191440,192013,192600,193075,193269,193856,194472,195021,195523,195988,195988,196611,197167,197776,198292,198732,199137,199555,199750
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,205599,205683,205750,205822,205903,205990,206069,206160,206270,206358,206452,206566,206649,206754,206878,206995,207079,207156,207254,207385,207509,207624,207764,207873,207970,208104,208245,208380,208532,208695,208839,208952,209111,209283,209463,209624,209817,209980,210152,210344
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,15382,15404,15404,15404,15425,15425,15462,15505,15516,15516,15516,15516,15516,15572,15618,15618,15618,15618,15705,15717,15744,15744,15819,15819,15819,15907,15929,15972,16035,16086,16086,16086,16299,16342,16426,16566,16712,16712,16712,16712
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,63567,63691,63775,63861,63930,64033,64126,64226,64301,64374,64433,64458,64487,64533,64583,64612,64654,64674,64724,64762,64815,64857,64875,64899,64913,64913,64940,64968,64985,64997,65011,65024,65033,65061,65080,65105,65130,65139,65144,65155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,NaN,Vietnam,14.058324,108.277199,0,2,2,2,2,2,2,2,2,2,6,6,8,8,8,10,10,13,13,14,15,15,16,16,16,16,16,16,16,16,16,16,16,16,16,16,...,877537,881522,884895,888940,892579,896174,900585,905477,910376,915603,921122,926720,932357,939463,946043,953547,961038,968684,976672,984805,992735,1000897,1009879,1018346,1026522,1035138,1045397,1055246,1065469,1075094,1084625,1094514,1104835,1143967,1155778,1168228,1181337,1197404,1210340,1224110
276,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,449980,450310,450310,450310,451459,451897,452272,452642,452997,452997,452997,453782,454172,454509,454799,455099,455099,455099,455689,455949,456186,456407,456632,456632,456632,457154,457390,457477,457729,457950,457950,457950,458448,458698,458956,459213,459479,459479,459479,460053
277,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,9635,9650,9662,9678,9694,9711,9728,9751,9773,9779,9791,9799,9806,9810,9831,9843,9858,9870,9883,9891,9902,9907,9912,9918,9918,9936,9936,9947,9950,9954,9955,9955,9961,9967,9972,9977,

In [ ]:
confirmed_global['Province/State'] = confirmed_global['Province/State'].fillna('Country')

In [ ]:
deaths_global['Province/State'] = deaths_global['Province/State'].fillna('Country')

In [ ]:
confirmed_global['Country/Region'].value_counts()

China                 34
Canada                16
France                12
United Kingdom        12
Australia              8
                      ..
Dominican Republic     1
Kosovo                 1
Ethiopia               1
Mongolia               1
Saint Lucia            1
Name: Country/Region, Length: 196, dtype: int64

In [ ]:
confirmed_us = confirmed_global[confirmed_global['Country/Region']=='US']
deaths_us = deaths_global[deaths_global['Country/Region']=='US']

In [ ]:
deaths_us

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,10/21/21,10/22/21,10/23/21,10/24/21,10/25/21,10/26/21,10/27/21,10/28/21,10/29/21,10/30/21,10/31/21,11/1/21,11/2/21,11/3/21,11/4/21,11/5/21,11/6/21,11/7/21,11/8/21,11/9/21,11/10/21,11/11/21,11/12/21,11/13/21,11/14/21,11/15/21,11/16/21,11/17/21,11/18/21,11/19/21,11/20/21,11/21/21,11/22/21,11/23/21,11/24/21,11/25/21,11/26/21,11/27/21,11/28/21,11/29/21
255,Country,US,40.0,-100.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,735332,737221,737725,737935,739401,741007,743076,744912,746636,746979,747189,748453,749725,751679,752861,754972,755385,755536,756791,758259,759916,760707,763209,763684,763828,765121,766410,768046,769326,771084,771417,771530,772762,774062,775724,776090,776349,776536,776639,778601


In [ ]:
confirmed_us_feat = confirmed_us[['Province/State','Country/Region','Lat','Long']]
deaths_us_feat = deaths_us[['Province/State','Country/Region','Lat','Long']]
deaths_us_time = deaths_us.drop(['Province/State','Country/Region','Lat','Long'],axis=1)
confirmed_us_time = confirmed_us.drop(['Province/State','Country/Region','Lat','Long'],axis=1)

In [ ]:
min_D = 7
max_D = 100


# the input should be a 1-D numpy array
def FindPeriod(arr):
    # initialize best period score
    best_score = float('inf')
    period = 0

    # D is the candidate period
    for d in range(min_D, max_D,7):

        # peaks locations
        p, _ = find_peaks(arr, distance=d)

        # valley locations
        v, _ = find_peaks(-arr, distance=d)

        # calculate interval lengths pd and vd from p and v
        pd = np.diff(p)
        vd = np.diff(v)

        # set current period d score
        cur_score = min(np.std(pd), np.std(vd)) / math.sqrt(d)

        if cur_score < best_score:
            best_score = cur_score
            period = d

    return period

    # convert data to 1-D numpy array
total_data=[]
total_data.append(np.diff(confirmed_us_time.to_numpy()).reshape(-1,1))

total_data.append(np.diff(deaths_us_time.to_numpy()).reshape(-1,1))


In [ ]:
period = [14]

In [3]:
input_size = 1
hidden_size = 16
num_layers = 1
num_classes = 1
batch_size = 128
#res=[]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.MSELoss().to(device)

In [2]:
def rescaler(data):
    #input_path = "/kaggle/input/mlproject/data-sets/KDD-Cup/data/001_UCR_Anomaly_35000.txt"
    #data = pd.read_csv(input_path,names=['value'])
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler.fit(data)
    return scaler



def dataPreprocessing(time_window, data, scaler):
    data = scaler.transform(data)
    data = data.tolist()
    dataX, dataY = [], []
    for i in range(len(data)-time_window-1):
        a = data[i:(i+time_window)]
        dataX.append(a)
        dataY.append(data[i + time_window])
    #print(dataX)
    return np.array(dataX), np.array(dataY)


def data_loader(data_x, data_y, batch_size):
    dataset = TimeDataset(data_x, data_y)
    return DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=4)


class TimeDataset(Dataset):
    def __init__(self, data_x, data_y):
        self.data_x = data_x
        self.data_y = data_y

    def __getitem__(self, idx):
        data_x = self.data_x[idx]
        data_y = self.data_y[idx]

        return data_x, data_y

    def __len__(self):
        return len(self.data_x)
    
    


    
    
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, device):
        super(LSTM, self).__init__()
        
        self.device = device
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        #self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        h_0 = h_0.cuda()
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        c_0 = c_0.cuda()
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        out = h_out.view(-1, self.hidden_size)
        
        
        #print(out.shape)
        return self.fc(out)
    
    
    
    
def train(train_loader,model,optimizer):
    num_epochs = 50
    count=0 
    for epochs in range(num_epochs):
        for i,(batch_x, batch_y) in enumerate (train_loader): 
            batch_x = batch_x.float()
            #print(batch_x.shape)
            batch_x = batch_x.to(device)
            batch_y = batch_y.float()
            batch_y = batch_y.to(device)
            #print(batch_x)
            outputs = model(batch_x)
            optimizer.zero_grad()   

            loss = criterion(outputs,batch_y)
            
            if epochs==0:
                last = loss.item()
            else:
                if (last-loss.item())/last<0.01:
                    count+=1
                elif count>0:
                    count=0
                #if count>2:
                    #break
            #backpropagation 
            loss.backward()
            optimizer.step()
            #print("Epoch: %d, loss: %1.5f" % (epochs, loss.item()))
            #iter+=1
        #if count>2:
            #break
        if epochs % 5 == 0:
            print("Epoch: %d, loss: %1.5f" % (epochs, loss.item()))

            
def evaluation(test_loader):
    res=[]
    for i,(batch_x, batch_y) in enumerate (test_loader): 
        batch_x = batch_x.float()
        batch_x = batch_x.to(device)
        batch_y = batch_y.float()
        batch_y = batch_y.numpy()
        with torch.no_grad():
            outputs = model(batch_x)
        outputs = outputs.cpu()
        outputs = outputs.numpy()
        res.append(outputs)
    res = np.concatenate(res,axis=0)
    return res

In [4]:
LSTM(num_classes, input_size, hidden_size, num_layers,device)

LSTM(
  (lstm): LSTM(1, 16, batch_first=True)
  (fc): Linear(in_features=16, out_features=1, bias=True)
)

In [ ]:
hit=0
count=0
res=[[],[]]
scalers=[]
for p in period:
  for idx in range(len(total_data)):
      scaler = rescaler(total_data[idx])
      scalers.append(scaler)
      train_x,train_y=dataPreprocessing(p, total_data[idx], scaler)
      data = scaler.transform(total_data[idx])
      #print(data.shape)
      test_x = np.array([data[-p:,:]])
      print(test_x.shape)
      test_y = np.array([0])
      
      train_loader = data_loader(train_x, train_y, batch_size)
      test_loader = data_loader(test_x, test_y, batch_size)
      
      model = LSTM(num_classes, input_size, hidden_size, num_layers,device)
      model = model.to(device)
      optimizer = torch.optim.Adam(model.parameters(), lr=0.001,betas=(0.9, 0.999), eps=1e-08,
                            weight_decay=0.001, amsgrad=False)
      
      train(train_loader,model,optimizer)
      
      
      test = torch.tensor(test_x).float().reshape(1,-1,1).to(device)
      #print(test_x.shape)
      with torch.no_grad():
          outputs = model(test)
      outputs = outputs.cpu()
      outputs = outputs.numpy()
      res[idx].append(outputs)
      #print(outputs.shape)

      for i in range(6):
        test_x = test_x[:,1:,:]
        test_x = np.concatenate((test_x,outputs.reshape(1,1,1)),axis=1)
        test = torch.tensor(test_x).float().reshape(1,-1,1).to(device)
        #print(test.shape)
        with torch.no_grad():
            outputs = model(test)
        outputs = outputs.cpu()
        outputs = outputs.numpy()
        res[idx].append(outputs)
      #res.append(outputs)
      #for i in range(6):

      #test_pred = evaluation(test_loader)
      
      #res = pd.DataFrame(np.absolute(test_y-test_pred))
      #res.to_csv('MSBD5002_Fall2021_Final_Data/Q2/',index=False)
      print()

(1, 14, 1)
Epoch: 0, loss: 0.23983
Epoch: 5, loss: 0.14328
Epoch: 10, loss: 0.07918
Epoch: 15, loss: 0.08164
Epoch: 20, loss: 0.07985
Epoch: 25, loss: 0.08017
Epoch: 30, loss: 0.07972
Epoch: 35, loss: 0.08472
Epoch: 40, loss: 0.08786
Epoch: 45, loss: 0.08752

(1, 14, 1)
Epoch: 0, loss: 0.21709
Epoch: 5, loss: 0.13634
Epoch: 10, loss: 0.09526
Epoch: 15, loss: 0.09562
Epoch: 20, loss: 0.09806
Epoch: 25, loss: 0.09934
Epoch: 30, loss: 0.09659
Epoch: 35, loss: 0.09548
Epoch: 40, loss: 0.09501
Epoch: 45, loss: 0.09484



In [ ]:
for i in range(len(res)):
  print([int(scalers[i].inverse_transform(j).tolist()[0][0]) for j in res[i]])

[91609, 91240, 90498, 89846, 88685, 92045, 94998]
[844, 817, 782, 759, 728, 730, 736]


In [ ]:
res[0]

[array([[0.8947974]], dtype=float32),
 array([[0.89323467]], dtype=float32),
 array([[0.8923458]], dtype=float32),
 array([[0.8916535]], dtype=float32),
 array([[0.8910987]], dtype=float32),
 array([[0.8906553]], dtype=float32),
 array([[0.8902894]], dtype=float32)]

**Task 2**

In [ ]:
confirmed_global.values

array([['Country', 'Afghanistan', 33.93911, ..., 157190, 157218, 157260],
       ['Country', 'Albania', 41.1533, ..., 199137, 199555, 199750],
       ['Country', 'Algeria', 28.0339, ..., 209980, 210152, 210344],
       ...,
       ['Country', 'Yemen', 15.552726999999999, ..., 9981, 9987, 9995],
       ['Country', 'Zambia', -13.133897, ..., 210138, 210143, 210150],
       ['Country', 'Zimbabwe', -19.015438, ..., 133836, 133951, 134226]],
      dtype=object)

In [ ]:
confirm_oct={}
oct = confirmed_global['10/1/21'] - confirmed_global['9/30/21']
confirmed_global = confirmed_global.values
for i in range(confirmed_global.shape[0]):
  if math.isnan(confirmed_global[i][2]) or math.isnan(confirmed_global[i][3]) or math.isnan(oct[i]):
    continue
  confirm_oct[(confirmed_global[i][2], confirmed_global[i][3])] = oct[i]
s = sum(oct)

In [ ]:
import folium
map = folium.Map(location=[32, 120], zoom_start=5)
for loc, c in confirm_oct.items():
    lat, lng = loc
    
    folium.CircleMarker(
            [lat, lng],
            radius=int(c/s),
      color='orange',
      fill_color='orange'
      ).add_to(map)

In [ ]:
map.save('5002final/wjy/map.html')